In [1]:

import numpy as np
import paddle
import torch
import os
from config_cvt.default import get_config
import torchvision.models as models
from cvt import generate_model
from cvt_torch import get_cls_model
model_name = 'CvT-13-224x224-IN-1k.pth'

config=get_config('./config_cvt/config.yaml')
paddle.set_device('cpu')
paddle_model = generate_model(config)
paddle_model.eval()


device = torch.device('cpu')
torch_model= get_cls_model(config)
torch_model.load_state_dict= torch.load(model_name,map_location=torch.device('cpu'))

torch_model = torch_model.to(device)
torch_model.eval()
file_debug=open('debug.txt','w')


merging config from ./config_cvt/config.yaml


In [2]:
def torch_to_paddle_mapping(torch_model,paddle_model):
    paddle_array=[]
    torch_array=[]
    for i,value in torch_model.state_dict().items():
        torch_array.append((i,value.shape))

    for i,value in paddle_model.state_dict().items():
        paddle_array.append((i,value.shape))

    j=0
    mapping=[]
    def equel(i,j):
        x:str=torch_array[i][0]
        y:str=paddle_array[j][0]
        if x==y:
            return True
        x=x.replace('.bn.','.1.')
        x=x.replace('.conv.','.0.')
        x=x.replace('.running_mean','._mean')
        x=x.replace('.running_var','._variance')
        if x==y:
            return True
        else:
            return False
    for i in range(len(torch_array)):
        if equel(i,j):
            mapping.append((torch_array[i][0],paddle_array[j][0]))
            j+=1
    if len(mapping)!=len(paddle_array):
        assert RuntimeError(f'mapping is not full,length of mapping is{len(mapping)},length of paddle_array is {len(paddle_array)}')
    return mapping

In [3]:
def convert(torch_model, paddle_model):
    def _set_value(th_name, pd_name, transpose=True):
        th_shape = th_params[th_name].shape
        pd_shape = tuple(pd_params[pd_name].shape) 

        file_debug.write(f'**SET** {th_name} {th_shape} **TO** {pd_name} {pd_shape}')
        file_debug.write('\n')
        if isinstance(th_params[th_name], torch.nn.parameter.Parameter):
            value = th_params[th_name].data.numpy()
        else:
            value = th_params[th_name].numpy()

        if len(value.shape) == 2 and transpose:
            value = value.transpose((1, 0))
        pd_params[pd_name].set_value(value)

    # 1. get paddle and torch model parameters
    pd_params = {}
    th_params = {}
    for name, param in paddle_model.named_parameters():
        pd_params[name] = param
    for name, param in torch_model.named_parameters():
        th_params[name] = param

    for name, param in paddle_model.named_buffers():
        pd_params[name] = param
    for name, param in torch_model.named_buffers():
        th_params[name] = param

    # 2. get name mapping pairs
    mapping = torch_to_paddle_mapping(torch_model,paddle_model)

    # 3. set torch param values to paddle params: may needs transpose on weights
    for th_name, pd_name in mapping:
        if th_name in th_params.keys(): # nn.Parameters
            _set_value(th_name, pd_name)
        else: # weight & bias
            th_name_w = f'{th_name}.weight'
            pd_name_w = f'{pd_name}.weight'
            _set_value(th_name_w, pd_name_w)

            if f'{th_name}.bias' in th_params.keys():
                th_name_b = f'{th_name}.bias'
                pd_name_b = f'{pd_name}.bias'
                _set_value(th_name_b, pd_name_b)

    return paddle_model


In [4]:
# convert weights
paddle_model = convert(torch_model, paddle_model)

# check correctness
x = np.random.randn(2, 3, 224, 224).astype('float32')
x_paddle = paddle.to_tensor(x)
x_torch = torch.Tensor(x).to(device)


In [5]:

out_torch = torch_model(x_torch)

tensor([[-1.1773, -0.6774, -0.7573, -1.3865, -0.0806, -0.5413,  0.1252, -0.3625,
          0.5254, -1.2392, -0.3171, -1.8559,  0.2038,  0.1487],
        [-0.1378, -1.5847, -0.6044,  0.6300, -1.3258, -2.8848, -1.9151, -2.0975,
         -0.7097, -1.5312,  0.7107,  1.9582, -2.4988,  1.5837],
        [-0.7401, -1.8067, -0.7765,  1.1386,  0.0215, -1.5315,  0.1295, -0.7527,
         -0.9552, -0.7494, -1.5780, -0.2543, -0.2353, -1.2985],
        [-1.5885, -0.7127,  0.3731, -1.5609,  2.2374, -0.0874, -0.2325, -0.7098,
          1.4446,  1.7226, -0.9152,  0.4734,  2.4674, -1.7448],
        [-0.6068,  0.4891,  0.1386, -1.0947, -0.3319,  2.3550,  0.2976,  0.1998,
          0.0323, -0.2740, -0.5816, -0.7407,  0.2390, -0.5712]],
       grad_fn=<SliceBackward0>)
tensor([-0.0670,  1.3199, -0.9743,  0.9939, -1.0772], grad_fn=<SliceBackward0>)
LayerNorm((384,), eps=1e-05, elementwise_affine=True)


In [6]:
out_paddle = paddle_model(x_paddle)

Tensor(shape=[5, 14], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-1.17422783, -0.67863894, -0.75977337, -1.38296187, -0.08453023,
         -0.54227579,  0.12237574, -0.36927927,  0.51981890, -1.23737621,
         -0.31640667, -1.86094725,  0.20431000,  0.14495680],
        [-0.13842249, -1.58719802, -0.60550010,  0.63185263, -1.33303630,
         -2.88414693, -1.91392028, -2.10013199, -0.71307099, -1.53035414,
          0.70702964,  1.95371425, -2.50320244,  1.57943487],
        [-0.74086303, -1.81200075, -0.77822685,  1.13973689,  0.02199420,
         -1.53317940,  0.13101058, -0.75560260, -0.95734674, -0.75512314,
         -1.57757545, -0.25386286, -0.23552293, -1.30169845],
        [-1.58804917, -0.71441519,  0.37347370, -1.56388450,  2.23283029,
         -0.08465839, -0.23329382, -0.71348417,  1.44600916,  1.71990228,
         -0.91834116,  0.46952003,  2.46424270, -1.74293053],
        [-0.60827297,  0.48698103,  0.13649459, -1.09423935, -0.33783397,
          2.

In [7]:




out_paddle = paddle_model(x_paddle)

out_torch = out_torch.data.cpu().numpy()
out_paddle = out_paddle.cpu().numpy()

file_debug.write(str(out_torch.shape))
file_debug.write('\n')
file_debug.write( str(out_paddle.shape))
file_debug.write('\n')
file_debug.write(str(out_torch[0, 0:100]))
file_debug.write('\n')
file_debug.write('========================================================')
file_debug.write('\n')
file_debug.write(str(out_paddle[0, 0:100]))
file_debug.write('\n')

assert np.allclose(out_torch, out_paddle, atol = 1e-5)

# save weights for paddle model
model_path = os.path.join(f'./{model_name}.pdparams')
paddle.save(paddle_model.state_dict(), model_path)
file_debug.write('all done')


Tensor(shape=[5, 14], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-1.17422783, -0.67863894, -0.75977337, -1.38296187, -0.08453023,
         -0.54227579,  0.12237574, -0.36927927,  0.51981890, -1.23737621,
         -0.31640667, -1.86094725,  0.20431000,  0.14495680],
        [-0.13842249, -1.58719802, -0.60550010,  0.63185263, -1.33303630,
         -2.88414693, -1.91392028, -2.10013199, -0.71307099, -1.53035414,
          0.70702964,  1.95371425, -2.50320244,  1.57943487],
        [-0.74086303, -1.81200075, -0.77822685,  1.13973689,  0.02199420,
         -1.53317940,  0.13101058, -0.75560260, -0.95734674, -0.75512314,
         -1.57757545, -0.25386286, -0.23552293, -1.30169845],
        [-1.58804917, -0.71441519,  0.37347370, -1.56388450,  2.23283029,
         -0.08465839, -0.23329382, -0.71348417,  1.44600916,  1.71990228,
         -0.91834116,  0.46952003,  2.46424270, -1.74293053],
        [-0.60827297,  0.48698103,  0.13649459, -1.09423935, -0.33783397,
          2.

AssertionError: 